# OpenStreetMap Data Case Study

## Map Area

I am interested to analyze map the city of Plano, Texas, since this is my hometown. I created a custom extract and downloaded it from Map Zen. Below is the link of the custom map extract.

https://mapzen.com/data/metro-extracts/your-extracts/530d0cc67c10


Unfortunately, defining the border for the extraction in Map Zen requires perfect rectangular, while it is not the case with the actual boundaries of the city. This problem will be handled later during analysis.

### Problem Encountered in the Map
After downloading the map for Plano, Texas and create a smaller subset of it, I noticed the following problems with the Map:

- Incorrect/typo in the street name. One example is "South Central Expessway". In this case the cleaning code updates the "Expessway" to "Expressway". Similar issue with "North Central Exressway" that the code also updates it to correct spelling.
- Inconsistent Street abreviation, for example Drive sometimes abbreviated as dr., dr or Dr (e.g. "McDermott Rd"). Instead of abbreviated that can be inconsistent, I prefer to have the addresses in full format. The cleaning code updates all abbreviated street name to actual name. Some examples here are: "St" changed to "Street", "Rd" to "Road", "Ave" to "Avenue" and "Dr" : "Drive".
- Street names in second level "k" tags puled from Tiger GPS data and divided into segments such as shown below. There are pros and cons to either keep or remove the entries from Tiger GPS, however I choose to not to remove them from the data.
		<tag k="tiger:cfcc" v="A41" />
		<tag k="tiger:tlid" v="103418948" />
		<tag k="tiger:county" v="Denton, TX" />
		<tag k="tiger:source" v="tiger_import_dch_v0.6_20070829" />


- Plano ZIP codes are 75023, 75024, 75025, 75074, 75075, 75093, 75094, 75026 and 75086. Other ZIP codes should be excluded from analysis. Whenever Plano specific data is required, the SQL query is filtered with above ZIP codes only. 
- Inconsistent and incorrect ZIP codes. For example, while all other addresses have 5 digits ZIP code, there are entries with 5 plus 4 ZIP codes format. Also there are addresses with ZIP code typo such as 75-75. These problems is rectified by updating the values in the database by using SQL update query (DML).
- As it will be shown later during analysis, there are less data points for Plano compared to neighbouring city Frisco. Moreover, geographically there was only minor part of Frisco included in the map export.  

## Incorrect/Typos and Inconsistent Abreviations in the Street Names

To make consistent street abbreviation, I use the update_name function below to check each street value and update it if it is in the mapping list. The function also fixes the typos identified earlier.


def update_name(name, mapping):

    m = street_type_re.search(name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            better_name = street_type_re.sub(mapping[street_type],name)
        else:
            better_name = name
    return better_name

## Zip Code
As mentioned earlier, the boundaries of the city is not perfet rectangular, thus the map data extract contains addresses from surounding city like Frisco, Allen, Parker, etc. Because of this, I use Zip Code or Postal Code to validate if the address is in Plano or not.
The valid zip codes for Plano are:
- 75013
- 75023
- 75024
- 75025
- 75074
- 75075
- 75093

The geographical mapping of Plano zip codes is available at this link:

http://www.zipmap.net/Texas/Collin_County/Plano.htm

Querying the data in SQL reveals one small problem though. There are 2 inconsistencies in zip code records. I could go back to the database preparation script and write the function to fix this issue, but since the problem is very limited, it makes more sense to update the records directly in the SQL as follow.

By querying Plano zip codes, one can see that zip code 75093 (on the west side of Plano) has the most data. This is not surprising as west of Plano is the area which is most developed in Plano.  

In [ ]:
sqlite> select comb.value, count(*) from (select * from nodes_tags union all select * from ways_tags) comb where comb.key="postcode" and comb.value in ("75013","75023","75024","75025","75074","75075","75093") group by comb.value order by count(*) desc;

value|count(*)
75093|321
75075|37
75074|35
75013|31
75023|26
75024|23
75025|23

However, the low count of other zip codes raises a question if the zip codes somehow got removed during processing. To verify this I check the original file.

$ ls -lh Plano.osm

-rwxrwxrwx@ 1 macbook  staff    69M Aug 19 22:58 Plano.osm

$ grep postcode Plano.osm|wc -l

     593

Based on this one can see that there are no many postcode data in the original osm file.

## Facilities

Plano is a small but vibrant city in North Texas. It keeps on growing and attracts new residents to move to Plano due to the facilities that the city has to offer. One of the considerations for people moving to a new place to live is the facilities. I am interested to know how many of each amenities are available in Plano map data.

In [ ]:
sqlite> select comb.value, count(*) from (select * from nodes_tags union all select * from ways_tags) comb 
where comb.key="amenity" and comb.id in (select distinct comb.id from 
(select * from nodes_tags union all select * from ways_tags) comb where comb.key="postcode" and comb.value in 
("75013","75023","75024","75025","75074","75075","75093")) group by comb.value order by count(*) desc;

value|count(*)
school|51
fast_food|20
restaurant|16
place_of_worship|6
pharmacy|5
post_office|5
bank|2
cinema|2
fuel|2
hospital|2
social_facility|2
bus_station|1
cafe|1
clinic|1
kindergarten|1
library|1
pub|1
townhall|1

Here we can see that schools is the prodominant amenities in Plano. This could be one of the reasons that Plano earns the reputation as a city with good education. 

One hidden problem here is that there are lack of datapoints for Plano. As an example there are 2nd bus station just opened few years back. Also there are five libraries in Plano instead of one shown in the result above.  


## Sort Cities by Count

As mentioned earlier, the data set also contains cities other than Plano. I am interested to see how many of the records are not for Plano.

In [ ]:
sqlite> select comb.value, count(*) from (select * from nodes_tags union all select * from ways_tags) comb where comb.key like "%city%"  group by comb.value order by count(*) desc;
value|count(*)

Frisco|9949
Plano|3067
Allen|41
Murphy|5
Richardson|5
Carrollton|4
Dallas|4
The Colony|3
McKinney|2
12|1
29|1
30|1
8|1
9|1
Allen TX|1
Allen, Texas|1
Dallas, Texas|1
Mckinney|1
Murphy, TX|1
Plano, TX|1
dallas|1
murphy|1

Surprisingly, the number of records for Frisco, a neighbouring city north of Plano, is far greater than it is for Plano. Although I tried to include as little as possible of Frico are when specifying boundaries for data export, it seems that there are far more data points reported for Frisco.

## Data Overview

Additional information regarding the data is presented below.

### File Size
    - Plano.osm      : 69 MB
    - PlanoDB2.db    : 39 MB
    - nodes.csv      : 27 MB
    - nodes_tags.csv : 2 MB
    - ways.csv       : 2.1 MB
    - ways_nodes.csv : 8.1 MB
    - ways_tags.csv  : 5.2 MB

### Number of Nodes and Ways



sqlite> select count(*) from nodes;

count(*)

315861

sqlite> select count(*) from ways;

count(*)

34106

### Number of Unique Users

sqlite> select count(distinct(comb.uid)) from (select uid from nodes union all select uid from ways) comb;

count(distinct(comb.uid))

386

### Top 10 Contributing Users

In [ ]:
sqlite> select comb.user, count(*) from (select user from nodes union all select user from ways) comb group by comb.user order by count(*) desc limit 10;

user|count(*)
Andrew Matheny_import|108609
woodpeck_fixbot|46531
Andrew Matheny|34545
Stephen214|31377
Saikrishna|16674
TaedeT|7883
Dami_Tn|7189
lemba|4619
AndrewMcQ|4602
Eagle1295|4201

One interesting fact is that user woodpeck_fixbot which is the 2nd top contributing user, is also a prominent contributors in other maps. The user name that is appended by "bot" may imply that the updates were done with automated tools.

In total, the top 5 users contributed to 61.22% of data points in the map.

In [ ]:
User             Count      Percentage

===============  =======    ==========

woodpeck_fixbot	 108,609 	31.03%

Andrew Matheny	 46,531 	13.30%

Stephen214	 34,545 	9.87%

Saikrishna   	 16,674 	4.76%

TaedeT      	  7,883 	2.25%

TOTAL	         214,242 	61.22%

## Improving Dataset

As mentioned earlier, there are issues concerning the quality of data. These issues need to be rectified in the form of data cleanup before it can be used for further analysis. While data clean up can be seen as an unavoidable and expected task in the data analysis work flow, it is always easier and takes less effort to work with data which is ready for analysis from the start.

Another problem identified is the lack of datapoints in the map for Plano. As Plano resident, I can see that there are places missing from the map. 

### Ideas to Improve Dataset

In order to increase the quality of the map while addressing above concerns, here are some ideas that I can think of.
- Standardize the input data. If the ZIP code is agreed to be in 5 digits format, then any input with other than 5 digits format should be rejected. Similarly, the type of street can be fixed into non abbreviated format that the users can choose from during the upload.
- Data validation and cross reference during upload. Data validation can be added into data upload process. For example if the data is for Plano address, it cannot use non Plano ZIP code. Google Map API is a good candidate for cross reference. 
- Gamification. To resolve the lack of datapoints issue, users can be encouraged to add more data by adding point scoring, competition with others, rules of play, etc. Gamification will increase the engagement with the product and creates and enhances the sense of ownership.

### Potential Implementation Issues
As with every improvement ideas, there are pitfals which are lurking and need to be addressed accordingly. Below are the possibilities.
- Standarization. With every effort of standarization, there comes the requirement to manage and keep the standarization to be accurate and up to date. This is additional effort that may or may not be feasible. Moreover, after standards are agreed upon, a decision needs to made regarding old data which does not conform to the standard.
- Data Validation. Cross reference with third party API or database requires legal and potentially commercial agreement. 
- Gamification may have finite effect on the users if the rules and incentives are not updated. That said, there must be away to entice the old members to keep on updating the map.
